# Lab 6: Topic Modelling

This lab explores topic modelling with Latent Dirichlet Allocation (LDA).

### Learning Outcomes

* Be able to apply Latent Dirichlet Allocation (LDA) using the Gensim library
* Know how to interpret the outputs of LDA for a specific document or topic 
* Know how to apply TF-IDF to improve the vector representations of the documents.

### Outline

* Implementing LDA Topic Modelling
* TF-IDF vectors
* Visualizing Topic Modelling Results
* Optional: HDP model as an alternative to LDA

### How To Complete This Lab

Read the text and the code then look for 'TODOs' that instruct you to complete some missing code. Look out for 'QUESTIONS' which you should try to answer before moving on to the next cell. Aim to work through the lab during the scheduled lab hours. To get help, you can talk to TAs or the lecturer during the labs, post questions to Blackboard (anonymously) or on Teams in the QA channel (with your name), or ask a question in the Wednesday live sessions. 

As you work through the notebooks, please make a note of any code that is unclear to you.

The labs *will not be marked*. However, they will prepare you for the coursework, so try to keep up with the weekly labs and have fun with the exercises! To understand what's going on inside the methods we use here, make sure to watch the lecture videos for the same week.

## Step 1: Load the Dataset

Topic modelling is uses unsupervised learning to extract the topics (represented as a probability distribution over words) that occur in a collection of documents. Let's load some data to apply topic modelling to from the 20 newsgroups dataset:

In [ ]:
from sklearn.datasets import fetch_20newsgroups

# We will use the 'train' split for learning the topics in an unsupervised manner:
newsgroups_train = fetch_20newsgroups(subset='train', shuffle=True, remove=('headers', 'footers', 'quotes'))

# We will apply our learned topic model to the 'test' split later on:
newsgroups_test = fetch_20newsgroups(subset='test', shuffle=True, remove=('headers', 'footers', 'quotes'))

Newsgroups are internet discussion groups, where users discuss a range of topics. Despite the name, the posts do not usually contain news. This dataset contains posts from 20 different newsgroups. Each newsgroup has a particular theme. We can view the list of newsgroups in the training split as follows:

In [ ]:
print(newsgroups_train.target_names)

When working with a new dataset, it is always good to play around with the data and see what we have, and how to find everything :) 
```newsgroups_train``` is a Python dictionary. Let's look at the keys and the types of objects stored in the dictionary:

In [ ]:
for i in newsgroups_train:
    print(i, type(newsgroups_train[i]))

The ```data``` item is a list of raw text documents. This is all we need for topic modelling, so we can ignore the ```target``` and ```DESCR``` keys for now.

The code below prints out the first post in ```data```:

In [ ]:
print('DOCUMENT 0:')
print(newsgroups_train.data[0])

## Step 2: Data Preprocessing

To apply topic modelling, we need to first preprocess the data. We will carry out the following steps using the same approach as previous labs:
* Tokenise the posts using NLTK's word_tokenize() function
* Remove non-word tokens and tokens with length less than 3 (likely to be numbers and punctuation that are not related to specific topics) and longer than 15 (probably URLs, codes, andbadly formatted tokens rather than proper words)
* Convert the tokens to lower case
* Remove stopwords: we have not used this step before; it removes tokens such as 'the' and 'a' that appear in a list of very common words, because these words do not tell us much about topics
* Lemmatize the tokens using WordNetLemmatizer to convert verbs to their root forms

We're going to introduce another library, [Gensim](https://radimrehurek.com/gensim/), which contains a lot of useful tools for topic modelling, text normalisation, and vector representations of words or documents (embeddings). For preprocessing, we will use the list of stopwords provided by Gensim. For topic modelling, removing stopwords can be particularly useful to remove noise and reduce the model complexity. Run the code below to preprocess the text:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer 
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS # find stopwords
import numpy as np

np.random.seed(400)  # We fix the random seed to ensure we get consistent results when we repeat the lab.

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in simple_preprocess(text) :  # Tokenize, remove very short and very long words, convert to lower case, remove words containing non-letter characters
        if token not in STOPWORDS:
            result.append(WordNetLemmatizer().lemmatize(token, 'v'))
            
    return result

# Create a list of preprocessed documents
processed = []
for doc in newsgroups_train.data:
    processed.append(preprocess(doc))

Now that we have finished the preprocessing, we need to construct the input for Gensim's topic modelling method. We do so by constructing a dictionary with word<->id mappings, then converting that into a bag of words, which will be the input to our model.

In [ ]:
from gensim.corpora import Dictionary

dictionary = Dictionary(processed) # construct word<->id mappings - it does it in alphabetical order
print(dictionary)

bow_corpus = [dictionary.doc2bow(doc) for doc in processed]

# 3. Latent Dirichlet Allocation (LDA)

Now we are ready to perform topic modelling using LDA. 

We are going to try 20 topics in the document corpus: the number of newsgroups is also 20, so perhaps we will find topics that correspond with the newsgroups. We will be running LDA using all CPU cores to parallelize and speed up model training.

Gensim provides the ```LdaModel``` class. When we construct an ```LdaModel``` object, some of the parameters we will be tweaking are:
   * *num_topics*, the number of requested latent topics to be extracted from the training corpus. <br>
   * *id2word*, a mapping from word ids (integers) to words (strings). It is used to determine the vocabulary size, as well as for debugging and topic printing. <br>
   * *workers*, the number of extra processes to use for parallelization. Uses all available cores by default. <br>


In [ ]:
from gensim.models import LdaModel

lda_model =  LdaModel(bow_corpus, 
                      num_topics=20, 
                      id2word=dictionary,                                    
                      passes=10,
                    ) 

TODO 3.1: run the code below to print out the topic distributions found by LDA. Can you find any topics that relate to specific newsgroups (newsgroups_train.target_names)? Can you find any other meaningful topics?

TODO 3.2: What do the values beside each word mean? 

In [ ]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

## Testing on unseen data

Now that we have a trained LDA mode, we can run it on a new, unseen document to get the breakdown of topics. Let's test the model on a document from the test set. First, get the raw document, then apply preprocessing:

In [ ]:
test_document_idx = 0
unseen_document = newsgroups_test.data[test_document_idx]
print(unseen_document)

print(f' This document is from newsgroup {newsgroups_test.target_names[newsgroups_test.target[test_document_idx]]}')

# Data preprocessing step for the unseen document - It is the same preprocessing we have performed for the training data
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for idx, count in bow_vector:
    print(f'{dictionary[idx]}: {count}')

Now, run our preprocessed document through the LDA model as follows to obtain $\boldsymbol{\theta}^d$, the topic distribution (topics with zero probability are not shown):

We can examine the word-topic distributions for the topics associated with this document:

In [ ]:
topic_distribution = lda_model[bow_vector]

for index, probability in sorted(topic_distribution, key=lambda tup: -1*tup[1]):
    print("Index: {}\nProbability: {}\t Topic: {}".format(index, probability, lda_model.print_topic(index, 5)))

## Visualising Topics 

Let's compare the topics for some training set documents in the same newsgroup.

TODO 3.3: Complete the code below to define a function to retrieve the topic distributions for 10 documents in the ```talk.politics.mideast``` newsgroup.

In [84]:
from gensim.matutils import any2sparse

def get_document_ids_in_newsgroup(newsgroup_name, newsgroups_data):
    # retrieve a list of document indexes for documents with this target_name
    doc_idxs = []
    for i, target_i in enumerate(newsgroups_data.target):
        if newsgroups_data.target_names[target_i] == newsgroup_name:
            doc_idxs.append(i)
            
    #print("There are {} documents in the newsgroup {}".format(len(doc_idxs), newsgroup_name))
            
    return doc_idxs


def get_topic_dists_in_newsgroup(newsgroup_name, lda_model, max_num_docs=10):
    doc_idxs = get_document_ids_in_newsgroup(newsgroup_name, newsgroups_train)
    
    # only use the first ten documents.
    if len(doc_idxs) > max_num_docs:
        doc_idxs = doc_idxs[:max_num_docs]
    print(doc_idxs)
    
    # Save each theta_d distribution to the list 'thetas':
    thetas = []
    
    for doc_idx in doc_idxs:
        ### COMPLETE THE CODE HERE
        
        # Get the document from newsgroups_train
        
        # Apply preprocessing to get a bag-of-words vector:
        
        # Use the LDA model to compute the topic distribution for this document:
        
        #######################
        
        thetas.append(theta_d)
    
    return thetas

Run the function above and print out the topic distributions:

In [ ]:
thetas = get_topic_dists_in_newsgroup('talk.politics.mideast', lda_model, max_num_docs=10)
print(thetas)

The text output is quite hard to read. We can improve on it by making a bar chart with a distinct colour for each topic. The function below can be used to plot a simple bar chart using the ```matplotlib``` library. This will help us see which documents discuss the same topics. The height of each bar is the probability for that topic within the document, according to our model.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# choose some colours for the topics
colours = ['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'teal', 'pink', 'purple',
           'orange', 'gray', 'lime', 'darkgreen', 'lightgray', 'navy', 'gold', 'crimson', 'darkgray', 'fuchsia']

def convert_theta_sparse_to_dense(theta_d_sparse, num_topics):
    theta_d = np.zeros(num_topics)  # an empty array
    
    # split the output from lda_model into two lists
    active_topics_in_d, probs = map(list, zip(*theta_d_sparse))
    
    # record the values in theta_d
    for i, topic in enumerate(active_topics_in_d):
        if topic >= num_topics:
            break
            
        theta_d[topic] = probs[i]
    
    return theta_d

# a function for producing a bar chart for a document
def plot_theta(thetas, d, num_docs, num_topics):
    plt.subplot(int(num_docs/3) + 1, 3, d+1)   # make a set of subplots inside a figure, with four subplots per row
    
    theta_d = convert_theta_sparse_to_dense(thetas[d], num_topics)
    
    # plot the results so that the same topics always occur at the same place along the x axis.
    plt.bar(x=np.arange(len(theta_d)), height=theta_d, color=colours, tick_label=np.arange(num_topics))

TODO 3.4: use the function to plot the topic distributions of the 10 documents we selected from the ```talk.politics.mideast``` newsgroup. What do you notice? Are there any topics the documents have in common? Any they do not? Refer back to the printed list of topics above to find the most common topic in this newsgroup.

In [ ]:
plt.figure(figsize=(20,8))

### WRITE YOUR OWN CODE HERE

############################

plt.show()

Now, let's look at how the topics we've found with LDA relate to the newsgroups (the targets in the training set).

The code below iterates over the newsgroups, computing the topic distribution for each document in that particular newsgroup. 

TODO 3.5: Complete the function below to compute the mean topic distribution, $\boldsymbol\theta$, for each newsgroup. Hint: use the ```mean``` function from Numpy. 

In [ ]:
# Create a numpy array which will store a matrix of values. Rows correspond to newsgroups and columns to LDA topics.
def get_newsgroups_mean_topics(lda_model, num_topics):
    
    # Create a matrix where each row corresponds to a newsgroup, and each column to a topic.
    # In each entry, we will save the mean probability of the topic for the documents in that newsgroup.
    mean_thetas = np.zeros((len(newsgroups_train.target_names), num_topics))

    print(mean_thetas.shape)
    for t, target_name in enumerate(newsgroups_train.target_names):
        # Obtain the thetas for the documents with this target name
        thetas_t_sparse = get_topic_dists_in_newsgroup(target_name, lda_model, max_num_docs=10)
        print(np.array(thetas_t_sparse))
        
        # convert the thetas to a dense vector format
        thetas_t = []
        for theta_d_t_sparse in thetas_t_sparse:
            if not theta_d_t_sparse:
                continue  # if it's empty
            thetas_d = convert_theta_sparse_to_dense(theta_d_t_sparse, num_topics)
            thetas_t.append(thetas_d)
            
        # compute the mean theta for this newsgroup and store it in mean_thetas 
        ### WRITE YOUR OWN CODE HERE

        
        ###########################

        #print(mean_thetas[t])
    return mean_thetas

We can now plot a matrix ```mean_thetas``` using ```matplotlib``` using the code below.

TODO 3.6: which LDA topics are common across many newsgroups? Are there any topics that are specific to particular newsgroups? 

In [ ]:
from IPython.display import clear_output

def plot_newsgroup_topic_matrix(model, num_topics):
    # Run the function we defined above to get the mean topic distributions:
    mean_thetas = get_newsgroups_mean_topics(model, num_topics)
    
    print(f'mean_thetas is a matrix of shape {mean_thetas.shape}')

    # Create a new figure
    plt.figure(figsize=(20,10))
    clear_output()

    # Plot the matrix as a 2-D grid, where colours represent the values.
    plt.imshow(mean_thetas)

    # Change the labels on the axes
    plt.yticks(range(len(newsgroups_train.target_names)), newsgroups_train.target_names )
    plt.xticks(range(num_topics))

    plt.show()
    
plot_newsgroup_topic_matrix(lda_model, num_topics)

# 4. Cosine Similarity and TF-IDF

For many tasks it is useful to compute similarity between documents, for example, if we want cluster documents into groups or to retrieve some documents that are similar to the one we are currently reading. In order to compute similarity or distance, we need to represent documents as numerical vectors. The code we have run so far produces bag-of-words vectors for each document, where each entry in vector is the count of word in the vocabulary.

Let's take a document from the 'rec.autos' newsgroup as a 'query' document and compare it to two others using cosine similarity. We'll take another one from 'rec.autos' and one from 'misc.forsale'. First let's show the documents and their bag of words vectors:

In [ ]:
# get our first document
doc_idxs = get_document_ids_in_newsgroup('rec.autos', newsgroups_train)
doc_idx_0 = doc_idxs[19]
print(newsgroups_train.data[doc_idx_0].strip())
query_doc = bow_corpus[doc_idx_0]
# show the bag of words vector in sparse format. In each pair of numbers, the first is the word ID and the second is the word count.
for idx, count in query_doc:
    print(f'{dictionary[idx]}: {count}')

In [ ]:
# get a comparison document from the same newsgroup
doc_idx_1 = doc_idxs[17]
comparison_doc_1 = bow_corpus[doc_idx_1]
print(newsgroups_train.data[doc_idx_1].strip())
# show the bag of words vector in sparse format. In each pair of numbers, the first is the word ID and the second is the word count.
for idx, count in comparison_doc_1:
    print(f'{dictionary[idx]}: {count}')

In [ ]:
# get another comparison document from another newsgroup
doc_idxs = get_document_ids_in_newsgroup('misc.forsale', newsgroups_train)
doc_idx_2 = doc_idxs[2]
comparison_doc_2 = bow_corpus[doc_idx_2]
print(newsgroups_train.data[doc_idx_2].strip())

# show the bag of words vector in sparse format. In each pair of numbers, the first is the word ID and the second is the word count.
for idx, count in comparison_doc_2:
    print(f'{dictionary[idx]}: {count}')

To compute the similarity between the documents, we use cosine similarity:

$$similarity<v_1, v_2> = \frac{v_1 \cdot v_2}{|| v_1 || \cdot || v_2 ||}$$


TODO 4.1: Which document do you expect to have higher similarity to the query? Run the code below and see if it meets your expectations.

In [ ]:
from gensim.matutils import cossim

cos_sim1 = cossim(query_doc, comparison_doc_1)
print(f'The cosine similarity between documents {doc_idx_0} and {doc_idx_1} is: {cos_sim1}')

cos_sim2 = cossim(query_doc, comparison_doc_2)
print(f'The cosine similarity between documents {doc_idx_0} and {doc_idx_2} is: {cos_sim2}')

The first and last documents share some common words, despite discussing different topics. We can alter our vector representations to focus more on keywords by using TF-IDF instead of bag of words.

TF-IDF is computed from two terms. First, the log of the term frequency:
$$ tf(t,d) = count(t,d)$$

We multiple the log term frequency by the inverse document frequencey, which is computed like this:

$$ idf(t) = \log_{2}\frac{N}{df(t)}$$

Gensim provides the TfidfModel class to compute TF-IDF vectors from our existing corpus object:

In [ ]:
from gensim.models import TfidfModel

tfidf_model = TfidfModel(bow_corpus)

TODO 4.2: Apply the tfidf_model to `query_doc`, `comparison_doc_1` and `comparison_doc_2` to obtain a TF-IDF vector for each document. Print the results. Hint: you can apply the model in the same was a you applied lda_model to a bow_vector for an unseen test document.

In [ ]:
#### WRITE YOUR OWN CODE HERE



#####

TODO 4.3: Compute the cosine similarity between the query document and the two comparison documents using the TF-IDF vectors. What do you notice about the results?

In [ ]:
### WRITE YOUR OWN CODE HERE



######

# 5. Optional: Hierarchical Dirichlet Process (HDP)

This section is optional if you want to learn more about HDP as an alternative to LDA. 

There is an implementation of the [HDP model provided by gensim](https://radimrehurek.com/gensim/models/hdpmodel.html). Instead of passing in a fixed number of topics, HDP will try to learn a good number of topics to fit the data.

OPTIONAL TODO 5.1: Refer to the documentation for HDP and train an HDP model. Hint: reuse the ```bow_corpus``` and ```dictionary``` as arguments in the same way that you did to construct the ```LdaModel``` object.

Use the trained HDP model to obtain mean topic distributions for each newsgroup in the test set with ```get_newsgroups_mean_topics()```. Plot the mean topic matrix as above and compare it to the results from LDA. Set alpha and gamma to 0.01.

In [ ]:
from gensim.models import HdpModel

### WRITE YOUR OWN CODE HERE


###

# print the word-topic distributions for 
for idx, topic in hdp_model.print_topics(20):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

The previous cell shows the first 20 topics. HDP learns the number of topics that are needed to model the dataset. It produces a global distribution over the topics. Topics with very low probability can be considered inactive:

In [ ]:
def plot_global_topic_weights(hdp_model):
    global_topic_weights = hdp_model.m_varphi_ss / np.sum(hdp_model.m_varphi_ss)

    plt.bar(np.arange(len(global_topic_weights)), global_topic_weights)
    plt.ylabel('Probability')
    plt.xlabel('Topic ID')
    
plot_global_topic_weights(hdp_model)

Run the code below to visualise the topics that HDP finds for each newsgroup:

In [ ]:
plot_newsgroup_topic_matrix(hdp_model, 20)

The alpha and gamma arguments to HdpModel control the 'concentration' of topics. Varying these parameters therefore affects the number of topics that HDP finds -- whether it tends towards many fine-grained topics, or few coarse-grained topics.

OPTIONAL TODO 5.2: Change the values of alpha and gamma for the HDP model and see what their effect is. Hint: to see noticable differences, change the values by a factor of 10. 

In [ ]:
### WRITE YOUR OWN CODE HERE

###

# print the word-topic distributions for 
for idx, topic in hdp_model.print_topics(20):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")
    
plot_global_topic_weights(hdp_model2)
    

In [ ]:
plot_newsgroup_topic_matrix(hdp_model2, 20)